In [1]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

# 1. Initialize the same embedding model used in your notebook
embeddings_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# 2. Load the saved FAISS index
# Note: You need the 'index' file and a way to map IDs back to your documents
vector_store = FAISS.load_local(
    "artifacts/movie_faiss_v2", 
    embeddings_model, 
    allow_dangerous_deserialization=True
)

# 3. Create a retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

d:\emerging\project\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from langchain.tools import tool

@tool
def search_movies(query: str) -> str:
    """Search the movie database for films based on plot, genre, or heart. 
    Use this when the user asks for recommendations."""
    # Assuming 'retriever' is defined from your FAISS vector store
    docs = retriever.invoke(query)
    return "\n\n".join([d.page_content for d in docs])

tools = [search_movies]

In [12]:
import os 
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY= os.getenv("GOOGLE_API_KEY")


In [13]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=GOOGLE_API_KEY,
    temperature=0.7,
    # thinking_budget=1000  # Optional: Enable reasoning tokens for 'pro' models
)

In [14]:
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware # New in v1.2

# Build the agent
agent = create_agent(
    model=llm, # Your gemma2:2b instance
    tools=tools,
    system_prompt=(
        "You are a movie recommendation assistant. "
        "If a user asks for a recommendation, use the search_movies tool. "
        "Always explain WHY you are recommending a specific movie."
    ),
    # Middleware handles long conversations so you don't run out of context
    middleware=[SummarizationMiddleware(model=llm, trigger_tokens=1500)]
)

In [16]:
# Instead of this:
# response = agent.invoke({"input": "I'm looking for a movie..."})

# Try this (The v1.2.6 State Pattern):
from langchain_core.messages import HumanMessage

response = agent.invoke({
    "messages": [HumanMessage(content="I'm looking for a heartwarming movie about dogs.")]
})

print(response["messages"][-1].content)

[{'type': 'text', 'text': 'I recommend the movie that tells the story of a fluffy, light-colored "mongrel" who is secretly cared for by a young boy named Colby. This heartwarming tale follows the dog as she helps her sick mother escape a cruel breeder and eventually becomes the famous Benji. It\'s a story of loyalty, love, and overcoming adversity, making it a perfect choice for a heartwarming dog movie.', 'extras': {'signature': 'CtMKAXLI2nxSD/Lv55rfNdwE55KMTTuhyQKydVTnGOwok46Fq6pt7Ol8obX74Q1JXYURaE1JxBEPxHXXLSPNquqT/Zqp1/G6NeErIsL9w89LGWIJWxIgtq9QP0FhtBjoi5ThWeADM6SY0RBrFT9fZmJaE6A2ZzGElKpLwE7SKB+4AUg7VFvjWjV+IdR5puYKO048guoZNLadNcJCiuA99qmx+yC98FGhG3T6qUahdB/W/qmTEo4yVHwn563RXxKCZCw3E+lKLR1BguUtWAbGjMh/Wx+b+gMhKVs8b+v6GzPQHhTPeSLNhpAs3/zD5OEF6MuBwM4MdM0pmKk8QCvS7vItQOJK3zo807DOeFD3zKRud9IrMxxoFWZ2H3e9bQgN4XMi5P1rCVES1ojHXX1TwWz1HpWi/48Znw9SiIWcqMrL4qfkzBtoydnBTm2UTBy7q89DfKeH0vMzplIwJlcOKIrTztOcX4J2XXmHU5xAlaBX5dmYymZ5PCULuaefcDUyuEueI/b1FBDT5jAJWMFmJJdsdLh7x0zi4s1QTikSvm8LRF6tVpIx0